In [ ]:
events = xml_processor.events['Name'].values
print(np.unique(events))

In [ ]:
event_type = 'Obstructive Apnea'
event_df = df_sleep_events[df_sleep_events['Name'].str.contains(event_type)]
event_df

# 2024-06-21 03:33:36.600	2024-06-21 03:33:54.830
start_datetime = datetime(2024, 6, 21, 4, 10, 00)
end_datetime = datetime(2024, 6, 21, 4, 11, 30) 

start_time = start_datetime
end_time = end_datetime

filtered_df = df.loc[start_time:end_time]
# Make a df subset with all the columns except: 
# 'ECG', 'Pleth', 'Movement Time (MT)', 'Wakefulness (W)', 'NREM Sleep Stage 1 (N1)', 
# 'REM Sleep', 'NREM Sleep Stage 2 (N2)', 'NREM Sleep Stage 3 (N3)'
columns_to_drop = ['Movement Time (MT)', 'Wakefulness (W)', 'NREM Sleep Stage 1 (N1)', 'REM Sleep', 'NREM Sleep Stage 2 (N2)', 'NREM Sleep Stage 3 (N3)']
df_subset = filtered_df.drop(columns=columns_to_drop)
print(df_subset.head())


In [ ]:
# Define events for specific event types (label value above threshold: 0)
# , df_subset['Limb Movement (Right)']
events3 = nk.events_find(df_subset[event_type], threshold=0, threshold_keep='above')
print(events3)

events4 = nk.events_find(df_subset[event_type], threshold=0, threshold_keep='below')
print(events4)

# Combine the arrays
combined_onset = np.concatenate((events3['onset'], events4['onset']))
combined_duration = np.concatenate((events3['duration'], events4['duration']))

# Sort the arrays based on the combined onset values
sorted_indices = np.argsort(combined_onset)
sorted_onset = combined_onset[sorted_indices]
sorted_duration = combined_duration[sorted_indices]
sorted_label = [str(i+1) for i in range(len(sorted_onset))]


# Create the merged dictionary
merged_events = {
    'onset': sorted_onset,
    'duration': sorted_duration,
    'label': sorted_label
}

print(merged_events)
plot = nk.events_plot(merged_events, df_subset[['ECG', 'Pleth']])


In [ ]:
# Process the signal
data_clean_rsp, info_rsp = nk.bio_process(
                                  rsp=df_subset["Pleth"],
                                  keep=df_subset[event_type],  
                                  sampling_rate=1024)

# Visualize some of the channels
data_clean_rsp[["RSP_Rate", event_type]].plot(subplots=True)